<a href="https://colab.research.google.com/github/Daniel-ASG/Web_Scraping/blob/main/deputados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obter os dados dentro de toda a página

In [7]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as  pd
from urllib.request import urlretrieve
import os

def trata_html(input):
    return ' '.join(input.split()).replace('> <', '><')

response = urlopen('https://www.camara.leg.br/deputados/quem-sao/resultado?search=&partido=&uf=&legislatura=56&sexo=')
html = response.read().decode('utf-8')
html = trata_html(html)
soup = BeautifulSoup(html, 'html.parser')

total_de_deputados = int(soup.find('div', {'class': 'busca-info__resultado busca-info__resultado--informado'}).getText().split(' ')[-2])
pages = int(total_de_deputados/25) + 1

url = []
cards = []


i=0
for page in range(pages):
    response = urlopen('https://www.camara.leg.br/deputados/quem-sao/resultado?search=&partido=&uf=&legislatura=56&sexo=&pagina=' + str(page+1))
    html = response.read().decode('utf-8')
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html.parser')

    paginas = soup.findAll('li', {'class': 'lista-resultados__item'})

    for pagina in paginas:
        url.append('https://www.camara.leg.br' + pagina.a.get('href'))

        response =  urlopen(url[i])
        html = response.read().decode('utf-8')
        html = trata_html(html)
        ficha = BeautifulSoup(html, 'html.parser')

        response =  urlopen(url[i]+'/biografia')
        html = response.read().decode('utf-8')
        html = trata_html(html)
        bio = BeautifulSoup(html, 'html.parser')


        # cria novo card para cada deputado
        card = {}

        infos_bio = bio.find('ul', {'class': 'informacoes-deputado'}).findAll('li')
        n=0
        for info in infos_bio:
            if n>2:
                card[info.getText().split(':')[0]] = info.getText().split(':')[1].strip()
            n+=1

        infos = ficha.find('ul', {'class': 'informacoes-deputado'}).findAll('li')
        for info in infos:
            card[info.getText().split(':')[0]] = info.getText().split(':')[1].strip()

        card['Nome'] = ficha.find('h2', {'class': 'nome-deputado'}).getText()
        card['Partido'] = ficha.find('span', {'class': 'foto-deputado__partido-estado'}).getText().split(' - ')[0]
        card['Estado'] = ficha.find('span', {'class': 'foto-deputado__partido-estado'}).getText().split(' - ')[1]

        if ficha.find('title').getText().strip().split(' ')[0] == 'Deputada':
            card['Sexo'] = 'Feminino'
        else:
            card['Sexo'] = 'Masculino'

        # acrescentar cota parlamentar (disponível/Gasto), Verba de gabinete, Salário, Imóvel funcional, Auxílio-moradia, Imóvel funcional
        
        # acrescenta o card deste deputado à lista cards
        cards.append(card)

        print(i)
        print(card)


        # dir = '/content/img/'
        # if not os.path.exists(dir):
        #     os.makedirs(dir)
        # urlretrieve(soup.img.get('src'), '/content/img/' + '_'.join(card['Nome'].split(' ')) + '.jpg')
        
        i+=1

dataset = pd.DataFrame(cards)
dataset.to_csv('/content/drive/MyDrive/Analise_de_dados/Web_Scraping/deputados.csv', sep=';', index=False, encoding='utf-8-sig')
dataset[['Nome', 'Nome Civil', 'Partido', 'Estado', 'Naturalidade', 'Sexo', 'Data de Nascimento', 'Profissões', 'Escolaridade', 'E-mail', 'Telefone', 'Endereço']]

0
{'Profissões': 'Empresário', 'Escolaridade': 'Superior Incompleto', 'Nome Civil': 'JOSE ABILIO SILVA DE SANTANA', 'E-mail': 'dep.abiliosantana@camara.leg.br', 'Telefone': '(61) 3215-5531', 'Endereço': 'Gabinete 531 - Anexo IV - Câmara dos Deputados', 'Data de Nascimento': '13/02/1965', 'Naturalidade': 'Salvador - BA', 'Nome': 'Abílio Santana', 'Partido': 'PL', 'Estado': 'BA', 'Sexo': 'Masculino'}
1
{'Profissões': 'Professor', 'Escolaridade': 'Superior', 'Nome Civil': 'PAULO SERGIO ABOU ANNI', 'E-mail': 'dep.abouanni@camara.leg.br', 'Telefone': '(61) 3215-5339', 'Endereço': 'Gabinete 339 - Anexo IV - Câmara dos Deputados', 'Data de Nascimento': '06/11/1966', 'Naturalidade': 'São Paulo - SP', 'Nome': 'Abou Anni', 'Partido': 'PSL', 'Estado': 'SP', 'Sexo': 'Masculino'}
2
{'Escolaridade': 'Superior', 'Nome Civil': 'ACÁCIO DA SILVA FAVACHO NETO', 'E-mail': 'dep.acaciofavacho@camara.leg.br', 'Telefone': '(61) 3215-5409', 'Endereço': 'Gabinete 409 - Anexo IV - Câmara dos Deputados', 'Data de

,Nome,Nome Civil,Partido,Estado,Naturalidade,Sexo,Data de Nascimento,Profissões,Escolaridade,E-mail,Telefone,Endereço
0,Abílio Santana,JOSE ABILIO SILVA DE SANTANA,PL,BA,Salvador - BA,Masculino,13/02/1965,Empresário,Superior Incompleto,dep.abiliosantana@camara.leg.br,(61) 3215-5531,Gabinete 531 - Anexo IV - Câmara dos Deputados
1,Abou Anni,PAULO SERGIO ABOU ANNI,PSL,SP,São Paulo - SP,Masculino,06/11/1966,Professor,Superior,dep.abouanni@camara.leg.br,(61) 3215-5339,Gabinete 339 - Anexo IV - Câmara dos Deputados
2,Acácio Favacho,ACÁCIO DA SILVA FAVACHO NETO,PROS,AP,Macapá - AP,Masculino,28/09/1983,NaN,Superior,dep.acaciofavacho@camara.leg.br,(61) 3215-5409,Gabinete 409 - Anexo IV - Câmara dos Deputados
3,Adolfo Viana,ADOLFO VIANA DE CASTRO NETO,PSDB,BA,Salvador - BA,Masculino,02/02/1981,Servidor público,Superior,dep.adolfoviana@camara.leg.br,(61) 3215-5911,Gabinete 911 - Anexo IV - Câmara dos Deputados
4,Adriana Ventura,ADRIANA MIGUEL VENTURA,NOVO,SP,São Paulo - SP,Feminino,06/03/1969,Administradora,Doutorado,dep.adrianaventura@camara.leg.br,(61) 3215-5802,Gabinete 802 - Anexo IV - Câmara dos Deputados
...,...,...,...,...,...,...,...,...,...,...,...,...
559,Zé Carlos,JOSÉ CARLOS NUNES JÚNIOR,PT,MA,São Luís - MA,Masculino,26/05/1955,Engenheiro,Superior,dep.zecarlos@camara.leg.br,(61) 3215-5543,Gabinete 543 - Anexo IV - Câmara dos Deputados
560,Zé Neto,JOSÉ CERQUEIRA DE SANTANA NETO,PT,BA,Feira de Santana - BA,Masculino,30/03/1964,Advogado,Pós-Graduação,dep.zeneto@camara.leg.br,(61) 3215-5585,Gabinete 585 - Anexo III - Câmara dos Deputados
561,Zé Silva,JOSÉ SILVA SOARES,SOLIDARIEDADE,MG,Iturama - MG,Masculino,11/05/1963,Agrônomo; Agricultor; Extensionista,Pós-Graduação,dep.zesilva@camara.leg.br,(61) 3215-5608,Gabinete 608 - Anexo IV - Câmara dos Deputados
562,Zé Vitor,JOSE VITOR DE RESENDE AGUIAR,PL,MG,Araguari - MG,Masculino,01/11/1984,NaN,Superior,dep.zevitor@camara.leg.br,(61) 3215-5525,Gabinete 525 - Anexo IV - Câmara dos Deputados


In [2]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as  pd
from urllib.request import urlretrieve
import os

def trata_html(input):
    return ' '.join(input.split()).replace('> <', '><')

response = urlopen('https://www.camara.leg.br/deputados/178833')
html = response.read().decode('utf-8')
html = trata_html(html)
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# acrescentar cota parlamentar (disponível/Gasto), Verba de gabinete, Salário, Imóvel funcional, Auxílio-moradia, Imóvel funcional

soup.find('section', {'class': 'gastos-deputado js-gastos-conteiner'})

<section class="gastos-deputado js-gastos-conteiner" id="gastos-section"><h2 class="titulo-interno">Gastos e recursos <span class="titulo-interno__ano"></span></h2></section>

In [10]:
pd.read_csv('/content/drive/MyDrive/Analise_de_dados/Web_Scraping/dataset.csv', sep=';')
# dataset[['Nome', 'Nome Civil', 'Partido', 'Estado', 'Naturalidade', 'Sexo', 'Data de Nascimento', 'Profissões', 'Escolaridade', 'E-mail', 'Telefone', 'Endereço']]

,Profissões,Escolaridade,Nome Civil,E-mail,Telefone,Endereço,Data de Nascimento,Naturalidade,Nome,Partido,Estado,Sexo,Filiação
0,Empresário,Superior Incompleto,JOSE ABILIO SILVA DE SANTANA,dep.abiliosantana@camara.leg.br,(61) 3215-5531,Gabinete 531 - Anexo IV - Câmara dos Deputados,13/02/1965,Salvador - BA,Abílio Santana,PL,BA,Masculino,NaN
1,Professor,Superior,PAULO SERGIO ABOU ANNI,dep.abouanni@camara.leg.br,(61) 3215-5339,Gabinete 339 - Anexo IV - Câmara dos Deputados,06/11/1966,São Paulo - SP,Abou Anni,PSL,SP,Masculino,NaN
2,NaN,Superior,ACÁCIO DA SILVA FAVACHO NETO,dep.acaciofavacho@camara.leg.br,(61) 3215-5409,Gabinete 409 - Anexo IV - Câmara dos Deputados,28/09/1983,Macapá - AP,Acácio Favacho,PROS,AP,Masculino,NaN
3,Servidor público,Superior,ADOLFO VIANA DE CASTRO NETO,dep.adolfoviana@camara.leg.br,(61) 3215-5911,Gabinete 911 - Anexo IV - Câmara dos Deputados,02/02/1981,Salvador - BA,Adolfo Viana,PSDB,BA,Masculino,NaN
4,Administradora,Doutorado,ADRIANA MIGUEL VENTURA,dep.adrianaventura@camara.leg.br,(61) 3215-5802,Gabinete 802 - Anexo IV - Câmara dos Deputados,06/03/1969,São Paulo - SP,Adriana Ventura,NOVO,SP,Feminino,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,Engenheiro,Superior,JOSÉ CARLOS NUNES JÚNIOR,dep.zecarlos@camara.leg.br,(61) 3215-5543,Gabinete 543 - Anexo IV - Câmara dos Deputados,26/05/1955,São Luís - MA,Zé Carlos,PT,MA,Masculino,José Carlos Nunes e Maria Dilma Duarte Nunes
560,Advogado,Pós-Graduação,JOSÉ CERQUEIRA DE SANTANA NETO,dep.zeneto@camara.leg.br,(61) 3215-5585,Gabinete 585 - Anexo III - Câmara dos Deputados,30/03/1964,Feira de Santana - BA,Zé Neto,PT,BA,Masculino,NaN
561,Agrônomo; Agricultor; Extensionista,Pós-Graduação,JOSÉ SILVA SOARES,dep.zesilva@camara.leg.br,(61) 3215-5608,Gabinete 608 - Anexo IV - Câmara dos Deputados,11/05/1963,Iturama - MG,Zé Silva,SOLIDARIEDADE,MG,Masculino,NaN
562,NaN,Superior,JOSE VITOR DE RESENDE AGUIAR,dep.zevitor@camara.leg.br,(61) 3215-5525,Gabinete 525 - Anexo IV - Câmara dos Deputados,01/11/1984,Araguari - MG,Zé Vitor,PL,MG,Masculino,NaN
